In [173]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import math

import chainer.functions as F
import chainer.links as L
from chainer import optimizers, serializers, Variable, dataset, Chain
from chainer.datasets import mnist
# from chainer.utils import type_check
# from chainer import function
import chainer

In [161]:
nz = 100 # Generatorの元乱数の次元
batchsize=100
n_epoch=10
n_train=20 # ?
image_save_interval = 5 # 画像の保存間隔

In [162]:
train, test = mnist.get_mnist(withlabel = False) # 取り敢えずmnist
# ネットワーク構造の都合上(32,32)にリサイズ
train_resized = np.zeros((train.shape[0], 32*32))
for i, t in enumerate(train):
    img = Image.fromarray(t.reshape(28, 28))
    img = img.resize((32,32))
    train_resized[i] = np.array(img).flatten()
train_resized = train_resized.reshape(train_resized.shape[0], 1, 32, 32).astype(np.float32)
# 0～1の範囲に正規化 ←　あってる？
train_resized = (train_resized - 0.5) / 0.5

In [163]:
# Generatorの定義
class Generator(Chain):
    def __init__(self):
        super(Generator, self).__init__()
        with self.init_scope():
            self.l0z = L.Linear(nz, 2*2*512, initialW=0.02*math.sqrt(nz))
            self.dc1 = L.Deconvolution2D(512, 256, 4, stride=2, pad=1, initialW=0.02*math.sqrt(4*4*512))
            self.dc2 = L.Deconvolution2D(256, 128, 4, stride=2, pad=1, initialW=0.02*math.sqrt(4*4*256))
            self.dc3 = L.Deconvolution2D(128, 64, 4, stride=2, pad=1, initialW=0.02*math.sqrt(4*4*128))
            self.dc4 = L.Deconvolution2D(64, 1, 4, stride=2, pad=1, initialW=0.02*math.sqrt(4*4*64))
            self.bn0l = L.BatchNormalization(2*2*512)
            self.bn0 = L.BatchNormalization(512)
            self.bn1 = L.BatchNormalization(256)
            self.bn2 = L.BatchNormalization(128)
            self.bn3 = L.BatchNormalization(64)
            
    def __call__(self, z):
        h = F.reshape(F.relu(self.bn0l(self.l0z(z))), (z.data.shape[0], 512, 2, 2))
        h = F.relu(self.bn1(self.dc1(h)))
        h = F.relu(self.bn2(self.dc2(h)))
        h = F.relu(self.bn3(self.dc3(h)))
        x = F.tanh(self.dc4(h))
        return x

In [164]:
class Discriminator(Chain):
    def __init__(self):
        super(Discriminator, self).__init__()
        with self.init_scope():
            self.c0 = L.Convolution2D(1, 64, 4, stride=2, pad=1, initialW=0.02*math.sqrt(4*4*1))
            self.c1 = L.Convolution2D(64, 128, 4, stride=2, pad=1, initialW=0.02*math.sqrt(4*4*64))
            self.c2 = L.Convolution2D(128, 256, 4, stride=2, pad=1, initialW=0.02*math.sqrt(4*4*128))
            self.c3 = L.Convolution2D(256, 512, 4, stride=2, pad=1, initialW=0.02*math.sqrt(4*4*256))
            self.l4l = L.Linear(2*2*512, 2, initialW=0.02*math.sqrt(6*6*512))
            self.bn0 = L.BatchNormalization(64)
            self.bn1 = L.BatchNormalization(128)
            self.bn2 = L.BatchNormalization(256)
            self.bn3 = L.BatchNormalization(512)
        
    def __call__(self, x):
        h = F.relu(self.c0(x))
        h = F.relu(self.bn1(self.c1(h)))
        h = F.relu(self.bn2(self.c2(h)))
        h = F.relu(self.bn3(self.c3(h)))
        l = self.l4l(h)
        return l

In [165]:
def train_dcgan(gen, dis):
    o_gen = optimizers.Adam(alpha=0.0002, beta1=0.5)
    o_dis = optimizers.Adam(alpha=0.0002, beta1=0.5)
    o_gen.setup(gen)
    o_dis.setup(dis)
    o_gen.add_hook(chainer.optimizer.WeightDecay(0.00001))
    o_dis.add_hook(chainer.optimizer.WeightDecay(0.00001))

    zvis = (np.random.uniform(-1, 1, (batchsize, nz))).astype(np.float32)
    
    for epoch in range(n_epoch):
        perm = np.random.permutation(n_train)
        sum_l_dis = np.float32(0)
        sum_l_gen = np.float32(0)
        
        for i in xrange(0, n_train, batchsize):
            # discriminator
            # 0: from dataset
            # 1: from noise

            #print "load image start ", i
            x2 = np.zeros((batchsize, 3, 96, 96), dtype=np.float32)
            for j in range(batchsize):
                try:
                    rnd = np.random.randint(len(dataset))
                    rnd2 = np.random.randint(2)

                    img = np.asarray(Image.open(StringIO(dataset[rnd])).convert('RGB')).astype(np.float32).transpose(2, 0, 1)
                    if rnd2==0:
                        x2[j,:,:,:] = (img[:,:,::-1]-128.0)/128.0
                    else:
                        x2[j,:,:,:] = (img[:,:,:]-128.0)/128.0
                except:
                    print 'read image error occured', fs[rnd]
            #print "load image done"
            
            # train generator
            z = Variable(xp.random.uniform(-1, 1, (batchsize, nz), dtype=np.float32))
            x = gen(z)
            yl = dis(x)
            L_gen = F.softmax_cross_entropy(yl, Variable(xp.zeros(batchsize, dtype=np.int32)))
            L_dis = F.softmax_cross_entropy(yl, Variable(xp.ones(batchsize, dtype=np.int32)))
            
            # train discriminator
                    
            x2 = Variable(cuda.to_gpu(x2))
            yl2 = dis(x2)
            L_dis += F.softmax_cross_entropy(yl2, Variable(xp.zeros(batchsize, dtype=np.int32)))
            
            #print "forward done"

            o_gen.zero_grads()
            L_gen.backward()
            o_gen.update()
            
            o_dis.zero_grads()
            L_dis.backward()
            o_dis.update()
            
            sum_l_gen += L_gen.data.get()
            sum_l_dis += L_dis.data.get()
            
            #print "backward done"

            if i%image_save_interval==0:
                pylab.rcParams['figure.figsize'] = (16.0,16.0)
                pylab.clf()
                vissize = 100
                z = zvis
                z[50:,:] = (xp.random.uniform(-1, 1, (50, nz), dtype=np.float32))
                z = Variable(z)
                x = gen(z, test=True)
                x = x.data.get()
                for i_ in range(100):
                    tmp = ((np.vectorize(clip_img)(x[i_,:,:,:])+1)/2).transpose(1,2,0)
                    pylab.subplot(10,10,i_+1)
                    pylab.imshow(tmp)
                    pylab.axis('off')
                pylab.savefig('%s/vis_%d_%d.png'%(out_image_dir, epoch,i))
                
        serializers.save_hdf5("%s/dcgan_model_dis_%d.h5"%(out_model_dir, epoch),dis)
        serializers.save_hdf5("%s/dcgan_model_gen_%d.h5"%(out_model_dir, epoch),gen)
        serializers.save_hdf5("%s/dcgan_state_dis_%d.h5"%(out_model_dir, epoch),o_dis)
        serializers.save_hdf5("%s/dcgan_state_gen_%d.h5"%(out_model_dir, epoch),o_gen)
        print 'epoch end', epoch, sum_l_gen/n_train, sum_l_dis/n_train

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(print 'read image error occured', fs[rnd])? (<ipython-input-165-d599f7896d15>, line 34)

In [172]:
cuda.get_device(0).use()

RuntimeError: CUDA environment is not correctly set up
(see https://github.com/chainer/chainer#installation).No module named 'cupy'

In [169]:
xp.random.uniform(1)

AttributeError: 'object' object has no attribute 'random'

In [183]:
zvis = (np.random.uniform(-1, 1, (batchsize, nz)))

In [184]:
zvis.shape

(100, 100)